<a href="https://colab.research.google.com/github/d823944/ML100-Days/blob/master/Homework/Day100_transfer_learning_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!
最後一天的作業請閱讀這篇非常詳盡的文章，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!
另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以參考

In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Dense, Dropout

from google.colab import files
src = list(files.upload().values())[0]
open('resnet_builder.py','wb').write(src)
from resnet_builder import resnet

Using TensorFlow backend.


Saving resnet_builder.py to resnet_builder (2).py


In [0]:
N_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 50

In [3]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# 建立 ResNet 模型
model = resnet(input_shape=(32, 32, 3), num_classes=N_CLASSES) 
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_____________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True)

from keras.callbacks import EarlyStopping
earlyStop = EarlyStopping(monitor="loss",
                          patience=10,
                          verbose=1)

from keras.callbacks import ReduceLROnPlateau
reduceLR = ReduceLROnPlateau(factor=0.3,
                             min_lr=1e-12,
                             monitor='loss',
                             patience=5,
                             verbose=1)

In [6]:
model.fit_generator(data_generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) / BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[earlyStop, reduceLR])

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
391/390 [==============================] - 42s 107ms/step - loss: 2.0218 - acc: 0.4541
Epoch 2/50
391/390 [==============================] - 34s 87ms/step - loss: 1.5802 - acc: 0.5832
Epoch 3/50
391/390 [==============================] - 34s 87ms/step - loss: 1.3917 - acc: 0.6307
Epoch 4/50
391/390 [==============================] - 34s 87ms/step - loss: 1.2705 - acc: 0.6672
Epoch 5/50
391/390 [==============================] - 34s 87ms/step - loss: 1.1693 - acc: 0.6962
Epoch 6/50
391/390 [==============================] - 34s 86ms/step - loss: 1.0970 - acc: 0.7204
Epoch 7/50
391/390 [==============================] - 34s 87ms/step - loss: 1.0344 - acc: 0.7381
Epoch 8/50
391/390 [==============================] - 34s 87ms/step - loss: 0.9861 - acc: 0.7549
Epoch 9/50
391/390 [==============================] - 34s 86ms/step - loss: 0.9513 - acc: 0.7643
Epoch 10/50
391/390 [==============================] - 34s 87ms/step - loss: 0

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9455831232070923
Test accuracy: 0.7924
